## Coefficient matrix for different basis sets 

Evaluate the rotation of different basis sets through rotated water molecules

In [ ]:
from collections import defaultdict
import json
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from ase.data import atomic_numbers, atomic_masses_iupac2016 
from pymatgen.core.structure import Molecule
from instance_mongodb import instance_mongodb_sei
import basis_set_exchange as bse
from minimal_basis.predata.predata_qchem import BaseQuantitiesQChem 
from pprint import pprint
from e3nn import o3
from store_structures import rotate_three_dimensions
import torch
%matplotlib inline

In [ ]:
db = instance_mongodb_sei(project="mlts")
collection = db.rotated_water_calculations
basis_sets = collection.distinct('orig.rem.basis')
data = defaultdict(lambda: defaultdict(list))

for basis_set in basis_sets:
    basis_info = bse.get_basis(basis_set, fmt='json', elements=[1, 8])
    basis_info = json.loads(basis_info)

    electron_shells = {k: basis_info['elements'][k]['electron_shells'] for k in basis_info['elements'].keys()}
    angular_momenta = {k: [shell['angular_momentum'] for shell in electron_shells[k]] for k in electron_shells.keys()}
    angular_momenta = {k: [item for sublist in angular_momenta[k] for item in sublist] for k in angular_momenta.keys()}

    for doc in collection.find({'orig.rem.basis': basis_set}):
        _alpha_coeff_matrix = doc['calcs_reversed'][0]['alpha_coeff_matrix']
        _alpha_eigenvalues = doc['calcs_reversed'][0]['alpha_eigenvalues']
        _alpha_fock_matrix = doc['calcs_reversed'][0]['alpha_fock_matrix']

        base_quantities_qchem = BaseQuantitiesQChem(
            fock_matrix=_alpha_fock_matrix,
            eigenvalues=_alpha_eigenvalues,
            coeff_matrix=_alpha_coeff_matrix,
        )

        _alpha_ortho_coeff_matrix = base_quantities_qchem.get_ortho_coeff_matrix()
        molecule = Molecule.from_dict(doc['orig']['molecule'])
        symbols = [site.specie.symbol for site in molecule.sites]
        atom_numbers = [atomic_numbers[symbol] for symbol in symbols]
        _basis_functions_orbital = []

        _irreps = ""
        for atom_number in atom_numbers:
            _angular_momenta = angular_momenta[str(atom_number)]
            _angular_momenta = np.array(_angular_momenta)
            _basis_functions = 2*_angular_momenta + 1
            for _basis_function in _basis_functions:
                if _basis_function == 1:
                    _basis_functions_orbital.extend(['s'])
                    _irreps += "+1x0e"
                elif _basis_function == 3:
                    _basis_functions_orbital.extend(['p', 'p', 'p'])
                    _irreps += "+1x1o"
                elif _basis_function == 5 and basis_set != '6-31g*':
                    _basis_functions_orbital.extend(['d', 'd', 'd', 'd', 'd'])
                    _irreps += "+1x2e"
                elif _basis_function == 5 and basis_set == '6-31g*':
                    _basis_functions_orbital.extend(['s', 'd', 'd', 'd', 'd', 'd'])
                    _irreps += "+1x0e+1x2e"

        _irreps = _irreps[1:]

        data[basis_set]['alpha_coeff_matrix'].append(_alpha_coeff_matrix)
        data[basis_set]['alpha_eigenvalues'].append(_alpha_eigenvalues)
        data[basis_set]['alpha_fock_matrix'].append(_alpha_fock_matrix)
        data[basis_set]['alpha_ortho_coeff_matrix'].append(_alpha_ortho_coeff_matrix)
        data[basis_set]['basis_functions_orbital'].append(_basis_functions_orbital)
        data[basis_set]['alpha_overlap_matrix'].append(base_quantities_qchem.overlap_matrix)
        data[basis_set]['irreps'].append(_irreps)
        data[basis_set]['angles'].append(doc['tags']['angles'])

for basis_set in basis_sets:
    for key in data[basis_set].keys():
        data[basis_set][key] = np.array(data[basis_set][key])

In [ ]:
basis_set = "def2-svp"
quantity = "alpha_ortho_coeff_matrix"

## $\alpha$ coefficient matrix

In [ ]:
fig = px.imshow(data[basis_set][quantity], animation_frame=0, range_color=[-1, 1])
labels = data[basis_set]['basis_functions_orbital'][0]
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Coefficient matrix from DFT calculation")
fig.show()


In [ ]:
new_coeff_matrix = []
calculated_coeff_matrix = data[basis_set][quantity]
original_coeff_matrix = calculated_coeff_matrix[0] 
for idx, angles in enumerate(data[basis_set]['angles']):
    irreps = data[basis_set]['irreps'][idx]
    irreps = o3.Irreps(irreps)

    rotation_matrix = rotate_three_dimensions(*angles) 
    rotation_matrix = torch.tensor(rotation_matrix, dtype=torch.float64)
    if idx == 0:
        rotation_matrix_0 = rotation_matrix
     
    rotation_matrix = rotation_matrix @ rotation_matrix_0.T

    D_matrix = irreps.D_from_matrix(rotation_matrix)
    D_matrix = D_matrix.detach().numpy()

    _new_coeff_matrix = np.zeros_like(original_coeff_matrix)
    for i in range(original_coeff_matrix.shape[1]):
        _new_coeff_matrix[:, i] = D_matrix @ original_coeff_matrix[:, i]# @ D_matrix.T

    new_coeff_matrix.append(_new_coeff_matrix)

new_coeff_matrix = np.array(new_coeff_matrix)
fig = px.imshow(new_coeff_matrix, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Rotated coefficient matrix")
fig.show()

In [ ]:
difference_matrices = np.abs(new_coeff_matrix) - np.abs(calculated_coeff_matrix)
fig = px.imshow(difference_matrices, animation_frame=0, range_color=[-1, 1])
fig.update_yaxes(ticktext=labels, tickvals=list(range(len(labels))))
fig.update_xaxes(ticktext=np.round(data[basis_set]['alpha_eigenvalues'][0], 3), tickvals=list(range(len(data[basis_set]['alpha_eigenvalues'][0]))))
fig.update_yaxes(title_text="Atomic orbitals")
fig.update_xaxes(title_text="Eigenvalues of molecular orbitals (Ha)")
fig.update_layout(title="Difference between rotated and calculated coefficient matrix")
fig.show()